In [1]:
import rasterio
from pathlib import Path

base = Path(r"E:\proyecto_agric_precision\Dataset de Imagenes\ortomosaicos_canales\drone_data\F10122025\BANDAS")

paths = [
    base / "20251210_B.tif",
    base / "20251210_G.tif",
    base / "20251210_R.tif",
    base / "20251210_RE.tif",
    base / "20251210_NIR.tif",
]

output_dir = Path(r"E:\proyecto_agric_precision\Dataset de Imagenes\ortomosaicos_canales\resultado")
output_dir.mkdir(parents=True, exist_ok=True)
output_path = output_dir / "2025_12_10_stack.tif"

for p in paths:
    if not p.exists():
        raise FileNotFoundError(f"No existe: {p}")

srcs = [rasterio.open(str(p)) for p in paths]

try:
    ref = srcs[0]
    profile = ref.profile.copy()

    # IMPORTANTE: forzar bloques válidos (múltiplos de 16)
    profile.update(
        count=5,
        BIGTIFF="YES",
        compress="deflate",
        tiled=True,
        blockxsize=256,
        blockysize=256
    )

    with rasterio.open(str(output_path), "w", **profile) as dst:
        # ventanas fijas (256x256) para asegurar poca RAM
        for ji, window in dst.block_windows(1):
            for i, src in enumerate(srcs, start=1):
                dst.write(src.read(1, window=window), i, window=window)

    print(f"Stacked image saved to {output_path}")
    print("CRS:", ref.crs)
    print("Shape (H, W):", (ref.height, ref.width))

finally:
    for s in srcs:
        s.close()


Stacked image saved to E:\proyecto_agric_precision\Dataset de Imagenes\ortomosaicos_canales\resultado\2025_12_10_stack.tif
CRS: EPSG:32631
Shape (H, W): (15790, 14944)
